## From a GrowClust output catalog, loop through the defined clusters and find the event within the cluster with the highest SNR.
## Save that event as an eqcorrscan template for matched-filter detection.
## Do this for each cluster!

In [101]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from obspy.core.event import read_events
import obspy
import eqcorrscan
import eqcorrscan.core.match_filter.template as template
from obspy.clients.fdsn.client import Client

#### Read in GrowClust output catalog:

In [2]:
outfile = '/Users/zoekrauss/GrowClust3D/endeavour/data/out/end_out_0835_2.grid3D.cat'

#### Get a list of distinct clusters:

In [56]:
cat = pd.read_csv(outfile,header=None,delim_whitespace=True)
cat.rename(columns={0:'yr',1:'mon',2:'day',3:'hr',4:'min',5:'sec',6:'evid',7:'latR',8:'lonR',9:'depR',\
                   10:'mag',11:'ev_serialID',12:'clusterID',13:'clustersize',14:'npair',15:'ndiffP',\
                   16:'ndiffS',17:'rmsP',18:'rmsS',19:'erh',20:'erz',21:'ert',22:'latC',23:'lonC',24:'depC'},inplace=True)

# Make a datetime column
cat = cat.astype({'yr':'int','mon':'int','day':'int','hr':'int','min':'int'})
cat['eventTime'] = cat[["yr","mon","day","hr","min"]].apply(lambda x: '-'.join(x.values.astype(int).astype(str)), axis="columns")
cat['eventTime'] = cat[['eventTime','sec']].apply(lambda x: '-'.join(x.values.astype(str)), axis="columns")
cat['eventTime']=pd.to_datetime(cat['eventTime'],format='%Y-%m-%d-%H-%M-%S.%f')


# Isolate events that actually got relocated
reloc = cat[~cat['rmsP'].isnull()]
clusters = reloc['clusterID'].unique()

#### Read in the catalog that was input to GrowClust:

In [19]:
cat = read_events('endquakes_2017.xml')
origin_ids = [str(p.resource_id)[-6:] for p in cat]

#### Loop through clusters and save the highest SNR event as eqcorrscan template:

In [111]:
temps = []
for c in clusters:
    
    # Pull in waveforms for one of the clusters
    clust = reloc[reloc['clusterID']==c]
    stream_list = []
    for id in clust['evid']:
        fid1 = 'sep2017_mseed/'+str(id)+'.mseed'
        strm = obspy.read(fid1).detrend()
         # Filter
        strm.filter('bandpass',freqmin=8,freqmax=35)
        stream_list.append(strm)
        
    # Cut them to all be the same length
    s_len = np.min([st[0].stats.npts for st in stream_list])
    cut_stream_list = []
    for st in stream_list:
        t1 = st[0].stats.starttime
        t2 = t1 + (s_len/200)
        st.trim(starttime=t1,endtime=t2,nearest_sample=False)
        if st[0].stats.npts > s_len:
            cut_stream_list.append(st[:s_len])
            
    # Find event with highest mean SNR (remember that the miniseeds are saved to have 5 seconds before the P-wave arrival)
    snr = []
    for st in cut_stream_list:
        snr_st = []
        for i,tr in enumerate(st):
            noise_amp = np.sqrt(np.mean(np.square(tr.data[0:900])))
            snr_st.append(max(tr.data) / noise_amp)
        snr.append(np.mean(snr_st))
    t_ind = np.argmax(snr)
    
    # Save that event as an eqcorrscan template
    temps.append(template.Template(name=str(c), st=cut_stream_list[t_ind], lowcut=8, highcut=35, samp_rate=200, prepick=5,process_length = 10,filt_order = 4))

In [112]:
# Write templates to a tribe
tribe = eqcorrscan.core.match_filter.tribe.Tribe(templates = temps)

# Now detect!
client = Client('IRIS')
t1 = obspy.UTCDateTime("2017-09-01-17")
t2 = obspy.UTCDateTime("2017-09-01-18")

party = tribe.client_detect(client,t1,t2,threshold=8, threshold_type='MAD',trig_int=1)

Error, routine incomplete, returning incomplete Party
Error: Template traces not all the same length
